In [ ]:
import base64
from groq import Groq
from pathlib import Path
from dotenv import load_dotenv
import requests
import os
import httpx
import sys
from langtrace_python_sdk import langtrace, inject_additional_attributes

load_dotenv(dotenv_path=Path.cwd().parent / ".env")
langtrace.init(api_key = os.getenv("LANGTRACE_API_KEY"),  write_spans_to_console=True) #toggle
groq_key = os.getenv("GROQ_API_KEY")


In [ ]:
parent_dir = Path("..").resolve()
sys.path.append(str(parent_dir))

print("Added to sys.path:", parent_dir)
from helpers.images import image_file_to_base64, image_file_to_data_url, get_latest_image

In [ ]:
# --- usage ---
CAPTURE_DIR = os.getenv('CAPTURE_DIRECTORY')
os.makedirs(CAPTURE_DIR, exist_ok=True)

image_path = get_latest_image(f"{CAPTURE_DIR}*")
IMAGE_DATA_URL = image_file_to_data_url(image_path)


client = Groq(api_key=groq_key)

completion = client.chat.completions.create(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": os.getenv("OPEN_AI_PROMPT"),
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": IMAGE_DATA_URL,
                    },
                },
            ],
        }
    ],
    temperature=1,
    max_completion_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")
